In [1]:
pip install -i https://pypi.tuna.tsinghua.edu.cn/simple/ pyecharts

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple/
Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests
from bs4 import BeautifulSoup

import time
import random
import re
import pandas as pd

url_list = ['https://book.douban.com/top250']

#適用for生成連結，讓其自動翻頁
for start in range(25, 226, 25):
    url = f'https://book.douban.com/top250?start={start}'
    url_list.append(url)


headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36'
}

book_list=[]
for url in url_list:

    sleep_time = random.uniform(0, 1)
    time.sleep(sleep_time)
    
    response = requests.get(url, headers=headers)
    #print(response.status_code)
    
    # 检查是否成功获取网页内容
    if response.status_code == 200:
        # 使用Beautiful Soup解析網頁內容
        soup = BeautifulSoup(response.text, 'html.parser')
        #print(soup.prettify())  #方便分析網頁結構   
        
        try:
            book_items = soup.find_all('tr', class_='item')  #獲取當頁圖書列表
            
            for book in book_items:
                # 提取書名
                book_title = re.sub(r'\s+', ' ',book.find('div', class_='pl2').find('a').text)
                # 提取作者
                author_info = book.find('p', class_='pl').text.split('/')[0].strip()                
                # 提取出版時間
                publication_date = book.find('p', class_='pl').text.split('/')[-2][0:5].strip() 
                
                # 提取評分
                rating = book.find('span', class_='rating_nums').text.strip()
                
                #提取出版社
                publishstore=book.find('p', class_='pl').text.split('/')[-3].strip() 
                
                #print(book_title,author_info,publication_date,rating,publishstore)
                book_list.append([book_title,author_info,publication_date,rating,publishstore])
            

        except:
            print(url,'解析异常')
    else:
        print(url,response.status_code)
        

column_names = ['書名', '作者', '出版時間', '評分','出版社']
df = pd.DataFrame(book_list, columns=column_names)
df.to_excel('Top2500.xlsx', index=False)

from pyecharts import options as opts
from pyecharts.charts import Bar

data = pd.read_excel('TOP2500.xlsx')


In [3]:
import pandas as pd
from pyecharts import options as opts
from pyecharts.charts import Bar

data = pd.read_excel('TOP2500.xlsx')
data.head(251)

,書名,作者,出版時間,評分,出版社
0,红楼梦,[清] 曹雪芹 著,1996.0,9.6,人民文学出版社
1,活着,余华,2012.0,9.4,作家出版社
2,1984,[英] 乔治·奥威尔,2010.0,9.4,北京十月文艺出版社
3,哈利·波特,J.K.罗琳 (J.K.Rowling),2008.0,9.7,人民文学出版社
4,三体全集 : 地球往事三部曲,刘慈欣,2012.0,9.5,重庆出版社
...,...,...,...,...,...
245,时间的秩序,[意] 卡洛·罗韦利,2019.0,8.9,湖南科学技术出版社
246,了不起的盖茨比,[美] 斯科特·菲茨杰拉德,2012.0,8.7,南海出版公司
247,夏洛的网,[美] E.B.怀特,2004.0,8.6,上海译文出版社
248,中国哲学简史,冯友兰,2004.0,8.9,新世界出版社


In [4]:
#將出版時間形成縱向排列的柱型圖
def getzoombar(data):
    year_counts = data['出版時間'].value_counts()
    year_counts.columns = ['出版時間', '数量']
    year_counts = year_counts.sort_index()
    c = (
        Bar()
        .add_xaxis(list(year_counts.index))
        .add_yaxis('出版數量', year_counts.values.tolist())
        .set_global_opts(
            title_opts=opts.TitleOpts(title='在榜圖書的出版年份分佈'),
            yaxis_opts=opts.AxisOpts(name='数量'),
            xaxis_opts=opts.AxisOpts(name='出版時間'),
            datazoom_opts=[opts.DataZoomOpts(), opts.DataZoomOpts(type_='inside')],)
        .render('distributions.html')
        )


In [5]:
getzoombar(data)

In [12]:
#將出版社形成橫向的柱型圖(top 15)
def getcountrybar(data):
    country_counts = data['出版社'].value_counts()
    country_counts.columns = ['出版社', '数量']
    country_counts = country_counts.sort_values(ascending=True)
    c = (
        Bar()
        .add_xaxis(list(country_counts.index)[-15:])
        .add_yaxis('出版社數量', country_counts.values.tolist()[-15:])
        .reversal_axis()
        .set_global_opts(
        title_opts=opts.TitleOpts(title='在榜圖書的出版社分佈'),
        yaxis_opts=opts.AxisOpts(name='国家/地区'),
        xaxis_opts=opts.AxisOpts(name='出版数量'),
        )
        .set_series_opts(label_opts=opts.LabelOpts(position="right"))
        .render('cbs3.html')
        )
getcountrybar(data)